In [1]:
from label.parallel_label import *

In [2]:
column_names=['src_ip','src_port','dst_ip','dst_port','proto','state','duration','sbytes','dbytes',
'sttl','dttl','sloss','dloss','service','Sload','Dload','Spkts','Dpkts','swin','dwin','stcpb','dtcpb',
'smeansz','dmeansz','trans_depth','res_bdy_len','Sjit','Djit','timestamp','Ltime','Sintpkt','Dintpkt',
'tcprtt','synack','ackdat','is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login',
'ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ ltm','ct_src_dport_ltm',
'ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','Label']

### Preparing provided csvs

Add column names to the csv of unsw. 

In [4]:
import os
import pandas as pd

def process_and_rename_columns(folder_path, new_column_names):
    # Ensure the column names list is not empty
    if not new_column_names:
        raise ValueError("New column names list cannot be empty.")

    # Iterate over all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers

                # Check if the number of new column names matches the number of columns in the CSV
                if len(new_column_names) != len(df.columns):
                    print(f"Skipping {file_path}: Number of new column names does not match number of columns.")
                    continue

                # Set the new column names
                df.columns = new_column_names

                # Save the DataFrame back to the same CSV file
                df.to_csv(file_path, index=False)  # Set index=False to avoid writing row numbers

                print(f"Processed {file_path}: Column names updated.")

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

folder_path = "/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"  # Replace with the path to your folder
process_and_rename_columns(folder_path, column_names)


/tmp/job.11310885/ipykernel_63165/1816870050.py:15: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers


Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv: Column names updated.
Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv: Column names updated.


/tmp/job.11310885/ipykernel_63165/1816870050.py:15: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None)  # Use header=None if there are no existing headers


Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv: Column names updated.
Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_4.csv: Column names updated.


Adding the attack_category column with 'Benign' for entries where it was empty and the label was 0

In [22]:
import os
import pandas as pd

def process_and_update_csvs(folder_path):
    # Iterate over all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Read the CSV file into a DataFrame
                df = pd.read_csv(file_path, low_memory=False)
                
                # Check if 'attack_cat' column exists
                if 'attack_cat' in df.columns:
                    # Fill empty strings and NaN values in 'attack_cat' with 'Benign'
                    df['attack_cat'].replace('', 'Benign', inplace=True)  # Replace empty strings
                    df['attack_cat'].fillna('Benign', inplace=True)      # Replace NaN values

                    # Show value counts for 'attack_cat'
                    print(f"File: {file_path}")
                    print(df['attack_cat'].value_counts())

                    # Save the DataFrame back to the same CSV file
                    df.to_csv(file_path, index=False)  # Set index=False to avoid writing row numbers

                    print(f"Processed and updated {file_path}")

                else:
                    print(f"Skipping {file_path}: 'attack_cat' column not found.")

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

# Example usage
folder_path = "/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"  # Replace with the path to your folder
process_and_update_csvs(folder_path)


File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
attack_cat
Benign            677786
Generic             7522
Exploits            5409
 Fuzzers            5051
Reconnaissance      1759
DoS                 1167
Backdoors            534
Analysis             526
Shellcode            223
Worms                 24
Name: count, dtype: int64
Processed and updated /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv
attack_cat
Benign              542576
Generic             118198
Exploits             16574
 Fuzzers              9137
DoS                   5642
 Reconnaissance       5582
Analysis               873
Backdoor               759
 Shellcode             593
Worms                   67
Name: count, dtype: int64
Processed and updated /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv
File: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv
attack_ca

### Extracting meta data from provided csvs (labeled csv)

In [5]:
if __name__ == '__main__':
    #set_start_method("fork")  # or "spawn" depending on your system

    # Example usage:
    folders = ["/scratch/user/syedwali/Datasets/UNSW/provided_CSVs"]

    # Step 1: Extract time ranges from CSV files in all folders using multiple cores
    meta_data = extract_time_ranges_from_csvs(folders, timestamp_column='timestamp', timezone='None', batch_size=5)

    # Print resulting metadata
    print(meta_data)

INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_4.csv: Min timestamp = 1424249984000, Max timestamp = 1424262068000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv: Min timestamp = 1424231099000, Max timestamp = 1424250009000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv: Min timestamp = 1421927377000, Max timestamp = 1421955842000
INFO:label.parallel_label:Processed /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_2.csv: Min timestamp = 1421955795000, Max timestamp = 1424231129000


                                           file_path  min_timestamp  \
0  /scratch/user/syedwali/Datasets/UNSW/provided_...  1421927377000   
1  /scratch/user/syedwali/Datasets/UNSW/provided_...  1424231099000   
2  /scratch/user/syedwali/Datasets/UNSW/provided_...  1421955795000   
3  /scratch/user/syedwali/Datasets/UNSW/provided_...  1424249984000   

   max_timestamp  
0  1421955842000  
1  1424250009000  
2  1424231129000  
3  1424262068000  


### Labeling

We are now labeling our processed CSV files using the labeled CSVs from UNSW. Since UNSW uses Unix timestamps, we are setting the timezone to None. Additionally, Unix timestamps have second-level precision, which we are considering. In the UNSW dataset, the labels are in the column named attack_cat, which we have also defined as function attributes.

In [26]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")
label_csvs(input_folder="/scratch/user/syedwali/Datasets/UNSW/processed", time_ranges_df=meta_data, timezone='None',num_workers=4,
          unit='sec',flowduration_col='duration',label_col='attack_cat')

Processing CSVs:   0%|          | 0/79 [00:00<?, ?it/s]

Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_1.csv
Reading: /scratch/user/syedwali/Datasets/UNSW/provided_CSVs/UNSW-NB15_3.csv


Processing CSVs:   1%|▏         | 1/79 [01:15<1:37:50, 75.27s/it]

Finished processing 11_1000000075_20240905-205247_processed.csv.
Label distribution:
label
Benign      17987
No Match       14
Name: count, dtype: int64
Finished processing 14_1028986624_20240905-211644_processed.csv.
Label distribution:
label
Benign      18339
No Match      141
Name: count, dtype: int64


Processing CSVs:   4%|▍         | 3/79 [01:17<25:39, 20.26s/it]  

...

Finished processing 19_processed.csv.
Label distribution:
label
Benign       20332
Exploits      7541
DoS           5154
 Fuzzers      2525
Generic        312
No Match         3
Name: count, dtype: int64


Processing CSVs:  99%|█████████▊| 78/79 [33:22<00:31, 31.46s/it]

Finished processing 9_1000001414_20240905-212854_processed.csv.
Label distribution:
label
Benign            16739
 Fuzzers           1274
Exploits             46
No Match             21
Reconnaissance       11
Name: count, dtype: int64


Processing CSVs: 100%|██████████| 79/79 [34:54<00:00, 26.51s/it]

Finished processing 16_processed.csv.
Label distribution:
label
Benign       36392
 Fuzzers      4108
Exploits       639
DoS            546
Generic        144
Name: count, dtype: int64
